In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import ipywidgets as widgets

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
#city_data_df.head()

In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
#Get the latitidue and longitude of each city
locations = city_data_df[["Lat","Lng"]]
#set any negative temperatures to 0F (shouldn't it be better to use Kelvin which is an absolute temp?)
city_data_df["Max Temp"] = city_data_df["Max Temp"].clip(lower=0)
#Get the max temperatures of each city
max_temp = city_data_df["Max Temp"]
# assign the gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating = False, 
                                    max_intensity=300,point_radius=4)
# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 89


In [10]:
temp_range_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp)]
temp_range_df = temp_range_df.dropna()
temp_range_df.head(5)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Minas,UY,2021-04-05 18:53:01,-34.3759,-55.2377,81.00,61,75,6.91
4,4,Port Blair,IN,2021-04-05 18:53:02,11.6667,92.7500,80.69,76,41,4.29
6,6,Vavoua,CI,2021-04-05 18:53:03,7.3819,-6.4778,84.16,56,50,3.83
8,8,Georgetown,MY,2021-04-05 18:53:03,5.4112,100.3354,82.99,83,40,2.30
11,11,Irituia,BR,2021-04-05 18:53:04,-1.7711,-47.4381,83.34,75,99,2.48


In [11]:
temp_range_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 2 to 569
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     254 non-null    int64  
 1   City        254 non-null    object 
 2   Country     254 non-null    object 
 3   Date        254 non-null    object 
 4   Lat         254 non-null    float64
 5   Lng         254 non-null    float64
 6   Max Temp    254 non-null    float64
 7   Humidity    254 non-null    int64  
 8   Cloudiness  254 non-null    int64  
 9   Wind Speed  254 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 21.8+ KB


In [12]:
hotel_df = temp_range_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Minas,UY,81.00,-34.3759,-55.2377,
4,Port Blair,IN,80.69,11.6667,92.7500,
6,Vavoua,CI,84.16,7.3819,-6.4778,
8,Georgetown,MY,82.99,5.4112,100.3354,
11,Irituia,BR,83.34,-1.7711,-47.4381,
12,Betare Oya,CM,80.82,5.6000,14.0833,
13,Castro,BR,73.20,-24.7911,-50.0119,
14,Sao Joao Da Barra,BR,80.60,-21.6403,-41.0511,
15,Mahebourg,MU,78.80,-20.4081,57.7000,
16,Pisco,PE,77.00,-13.7000,-76.2167,


In [21]:
def find_hotel(row) :
    lat = row["Lat"]
    lng = row["Lng"]
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key,
        "location": f"{lat},{lng}"}
    r = requests.get(base_url,params).json()
    try :
        row["Hotel Name"] = r["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping")
    return row

hotel_df = hotel_df.apply(find_hotel,axis=1)
hotel_df.head()

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Minas,UY,81.00,-34.3759,-55.2377,Hotel Minas
4,Port Blair,IN,80.69,11.6667,92.7500,Welcomhotel Bay Island Port Blair
6,Vavoua,CI,84.16,7.3819,-6.4778,Hotel Le Peuple
8,Georgetown,MY,82.99,5.4112,100.3354,Cititel Penang
11,Irituia,BR,83.34,-1.7711,-47.4381,Hotel Irituia


In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))